有了条件等待队列,和cas实现无锁等待的基础后,就可以进入AQS的学习,这是所有jdk同步器的基类

#### AQS javadoc
1. AQS提供一个框架,来实现依靠 FIFO 等待队列的锁和各种同步器(semaphores, events等等). AQS 是这些同步器的基类, 仅用一个 `atomic int` 型变量表示状态. 子类要声明方法去修改这个 `state` 变量的值, 来表示对象被被获取和释放的状态. 子类可以声明其他状态相关的字段, 但只有通过 `getState()`, `setState()` 和 `compareAndSetState()` 3个方法追踪的 state 变量才和同步相关

2. AQS 的子类,应该定义成非 public 的内部类, 来被外部类暴露的方法调用     
   如 `ReentrantLock#Sync` 是以内部类形式出现的, Sync 才是 AQS 的实现
   
3. AQS 同时支持排他模式和共享模式. 
    * 排他模式: exclusive mod. 在排他模式下accquire 成功,则其他线程无法再 accquire
    * 共享模式: shared mode. 共享模式下,多个线程都可以 accquire 成功.     
  但AQS 并不识别这两种模式, 执行的是同一套操作. 比如, 一个线程在共享模式下 accquire 成功, 其他等待线程仍然要去判断是否能 accquire. 通常 AQS 的子类要么是共享模式, 要么是排他模式, 实现其中一种即可. 只有 ` ReadWriteLock` 同时实现了 AQS 的共享和排他模式

4. AQS 有一个 public 的内部类 `ConditionObject`, 被用作实现了 Condition 接口的子类. `ConditionObject` 只支持排他模式, 是否是排他模式由 AQS 的子类复写方法 `isHeldExclusively()` 返回, 该方法表示当前线程是否在排他模式下获取了同步器. 

5. 同步器实现 AQS 以下五选三的方法修改 state 变量的值即可 (使用 getState,setState,compareAndSetState 修改 state 变量) (只有ReadWriteLock内部的同步器5个都实现) 
    * 排他模式: tryAcquire()  tryRelease()
    * 共享模式: tryAcquireShared()  tryReleaseShared()
    * 判断排他还是共享模式: isHeldExclusively()     
    
  这三个方法实现的要简短, 要线程安全,且是非阻塞的修改变量(cas修改)

6. 尽管AQS内部依靠一个 cas 实现的 FIFO 队列, 但他的线程 accquire 策略却不强制要求是 FIFO 的. Acquire, Release 实现的语义大致如下:
    ```java
    Acquire:
        while (!tryAcquire(arg)) {
           <em>enqueue thread if it is not already queued(讲线程插入 FIFO 队列)</em>;
           <em>possibly block current thread (阻塞当前线程)</em>;
        }
    Release:
        if (tryRelease(arg))
           <em>unblock the first queued thread(解锁FIFO队列中的第一个线程)</em>;

    /** 共享模式下逻辑相似,只是会级联调用 signal */
    ```
7. 因为在线程进入 FIFO 队列前, 要先 tryAccquire(), 新线程可能和FIFO队列头的线程产生冲撞, 先于队列头的线程 accquire 成功, 导致线程的 accquire 非公平. 所以公平模式的同步器, tryAcquire() 会返回 false 如果 `hasQueuedPredecessors()` 返回true (hasQueuedPredecessors 经常在公平模式下声明, 表示FIFO 队列中是否有等待线程. ReentrantLock的`FairSync`和`NonFairSync`的 tryaccquire()内容, 只是多了一个 hasQueuedPredecessors 判断). 即公平模式只有在FIFO队列为空时才能获取同步器成功, 否则就会用 cas 无线重试的方式等待队列中的线程全部执行完. 导致公平模式效率低. 
  例子: 一个不可重入的排它锁, state=0 表示未持有锁, state=1表示持有锁, 他没有严格要求同一线程可以反复持有锁
```java
 class Mutex implements Lock, java.io.Serializable {
    // Our internal helper class
    private static class Sync extends AbstractQueuedSynchronizer {
        // Reports whether in locked state
        protected boolean isHeldExclusively() {
            return getState() == 1;
        }

        // Acquires the lock if state is zero
        public boolean tryAcquire(int acquires) {
            assert acquires == 1; // Otherwise unused
            if (compareAndSetState(0, 1)) {
                setExclusiveOwnerThread(Thread.currentThread());
                return true;
            }
            return false;
        }

        // Releases the lock by setting state to zero
        protected boolean tryRelease(int releases) {
            assert releases == 1; // Otherwise unused
            if (getState() == 0) throw new IllegalMonitorStateException();
            setExclusiveOwnerThread(null);
            setState(0);
            return true;
        }

        // Provides a Condition
        Condition newCondition() { return new ConditionObject(); }

        // Deserializes properly
        private void readObject(ObjectInputStream s)
                throws IOException, ClassNotFoundException {
            s.defaultReadObject();
            setState(0); // reset to unlocked state
        }
    }

    // The sync object does all the hard work. We just forward to it.
    private final Sync sync = new Sync();

    public void lock()                { sync.acquire(1); }
    public boolean tryLock()          { return sync.tryAcquire(1); }
    public void unlock()              { sync.release(1); }
    public Condition newCondition()   { return sync.newCondition(); }
    public boolean isLocked()         { return sync.isHeldExclusively(); }
    public boolean hasQueuedThreads() { return sync.hasQueuedThreads(); }
    public void lockInterruptibly() throws InterruptedException {
        sync.acquireInterruptibly(1);
    }
    public boolean tryLock(long timeout, TimeUnit unit)
            throws InterruptedException {
        return sync.tryAcquireNanos(1, unit.toNanos(timeout));
    }
}
```

#### Node(Simple FIFO Queue) JavaDoc
AQS 内部的 cas 版 fifo 队列,是简易版的 cas 实现, 不是 `ConcurrentLinkedQueue` 那种完全的算法实现